In [1]:
!pip install langchain crewai crewai_tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.3/265.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 109.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 573.1/573.1 kB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.3/211.3 kB 19.2 MB/s 

In [2]:
from google.colab import userdata
userdata.get('OPENROUTER_API_KEY')

'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'

In [3]:
from langchain.chat_models import ChatOpenAI
import os

# Set API Key
os.environ["OPENROUTER_API_KEY"] = userdata.get('OPENROUTER_API_KEY')

# Configure LLM with CORRECT headers
llm = ChatOpenAI(
    model="openrouter/meta-llama/llama-3-70b-instruct",
    openai_api_base="https://openrouter.ai/api/v1",
    temperature=0.7,
    openai_api_key=userdata.get('OPENROUTER_API_KEY'),  # Still pass this
    model_kwargs={
        "headers": {
            "Authorization": f"Bearer {userdata.get('OPENROUTER_API_KEY')}",  # 🔑 Critical fix
            "HTTP-Referer": "https://your-site.com",  # Required
            "X-Title": "CrewAI Project"  # Required
        }
    }
)

<ipython-input-3-5556521737b1>:8: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [8]:
from crewai import Task, Agent, Crew

In [4]:
from crewai import Agent

planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    llm=llm,
    allow_delegation=False,
	verbose=True
)

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    llm=llm,
    allow_delegation=False,
    verbose=True
)

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    llm=llm,
    allow_delegation=False,
    verbose=True
)

In [9]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

In [10]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

In [11]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

In [12]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

In [13]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: a7d9653c-d49e-47fe-bae2-2cecefb4a9be                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 93d4b85a-d86a-4517-b874-706e9fbac295
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 93d4b85a-d86a-4517-b874-706e9fbac295
       Status: Executing Task...
    └── 🤖 Agent: Content Planner
            Status: In Progress

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


🚀 Crew: crew
└── 📋 Task: 93d4b85a-d86a-4517-b874-706e9fbac295
       Status: Executing Task...
    └── 🤖 Agent: Content Planner
            Status: In Progress
        └── 🧠 Thinking...

...



# Agent: Editor
## Final Answer: 
**Demystifying Artificial Intelligence: Trends, Applications, and Future Possibilities**

**I. Introduction**
---------------

Artificial Intelligence (AI) has been a buzzword in the tech industry for decades, but its true potential and significance are still not fully understood by many. At its core, AI refers to the development of computer systems that can perform tasks that typically require human intelligence, such as learning, problem-solving, and decision-making. From virtual assistants like Siri and Alexa to self-driving cars and personalized product recommendations, AI is transforming the way we live and work. In this article, we'll delve into the latest trends, applications, and future possibilities of AI, and explore why understanding this technology is crucial in today's world.

The history of AI dates back to the 1950s, when computer scientists like Alan Turing and Marvin Minsky first proposed the concept of machine learning. Since then, 

🚀 Crew: crew
├── 📋 Task: 93d4b85a-d86a-4517-b874-706e9fbac295
│      Assigned to: Content Planner
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Planner
│           Status: ✅ Completed
├── 📋 Task: e7879ca4-617c-4432-879b-acada47bea4f
│      Assigned to: Content Writer
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Writer
│           Status: ✅ Completed
└── 📋 Task: 6d422487-2fcd-46dd-a399-d0348a956b5b
       Status: Executing Task...
    └── 🤖 Agent: Editor
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 93d4b85a-d86a-4517-b874-706e9fbac295
│      Assigned to: Content Planner
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Planner
│           Status: ✅ Completed
├── 📋 Task: e7879ca4-617c-4432-879b-acada47bea4f
│      Assigned to: Content Writer
│      Status: ✅ Completed
│   └── 🤖 Agent: Content Writer
│           Status: ✅ Completed
└── 📋 Task: 6d422487-2fcd-46dd-a399-d0348a956b5b
       Assigned to: Editor
       Status: ✅ Completed
    └── 🤖 Agent: Editor
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 6d422487-2fcd-46dd-a399-d0348a956b5b                                                                     │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: a7d9653c-d49e-47fe-bae2-2cecefb4a9be                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [15]:
print(result)

**Demystifying Artificial Intelligence: Trends, Applications, and Future Possibilities**

**I. Introduction**
---------------

Artificial Intelligence (AI) has been a buzzword in the tech industry for decades, but its true potential and significance are still not fully understood by many. At its core, AI refers to the development of computer systems that can perform tasks that typically require human intelligence, such as learning, problem-solving, and decision-making. From virtual assistants like Siri and Alexa to self-driving cars and personalized product recommendations, AI is transforming the way we live and work. In this article, we'll delve into the latest trends, applications, and future possibilities of AI, and explore why understanding this technology is crucial in today's world.

The history of AI dates back to the 1950s, when computer scientists like Alan Turing and Marvin Minsky first proposed the concept of machine learning. Since then, AI has evolved rapidly, with signifi

In [14]:
from IPython.display import Markdown
Markdown(result)

TypeError: Markdown expects text, not CrewOutput(raw="**Demystifying Artificial Intelligence: Trends, Applications, and Future Possibilities**\n======================================================================\n\n**I. Introduction**\n---------------\n\nArtificial Intelligence (AI) has been a buzzword in the tech industry for decades, but its true potential and significance are still not fully understood by many. At its core, AI refers to the development of computer systems that can perform tasks that typically require human intelligence, such as learning, problem-solving, and decision-making. From virtual assistants like Siri and Alexa to self-driving cars and personalized product recommendations, AI is transforming the way we live and work. In this article, we'll delve into the latest trends, applications, and future possibilities of AI, and explore why understanding this technology is crucial in today's world.\n\nThe history of AI dates back to the 1950s, when computer scientist